# Online Inference

This notebook simulates an online inference pipeline by publishing data to a Kinesis stream which is picked up by an associated Lambda function and this Lambda function joins this data with additional data from an online feature store and then invokes a SageMaker endpoint to get inference in real-time.


In [174]:
from pathlib import Path
import pandas as pd
import logging
import boto3
import json
import sys
import os

In [175]:
# import from a different path
path = Path(os.path.abspath(os.getcwd()))
package_dir = f'{str(path.parent)}/utils'
print(package_dir)
sys.path.insert(0, package_dir)
import utils

/home/ec2-user/SageMaker/feature-store-expedia/utils


## Setup logging

In [176]:
logger = logging.getLogger('__name__')
logging.basicConfig(format="%(asctime)s,%(filename)s,%(funcName)s,%(lineno)s,%(levelname)s,p%(process)s,%(message)s", level=logging.INFO)       

## Global constants

In [177]:
# global constants
STACK_NAME = "expedia-feature-store-demo-v2"
LOCAL_DATA_DIR = "../data"
DESTINATION_FEATURES = "pc1,pc2,pc3"
PREDICTED_VARIABLE = "hotel_cluster_predicted"
RECORDS_TO_STREAM = 5
PK = 'user_id' # partition key for kinesis stream

## Setup config variables

In [178]:
# read output variables from cloud formation stack, these will be used as parameters throughout
# the code
data_bucket_name = utils.get_cfn_stack_outputs(STACK_NAME, 'DataBucketName')
model_bucket_name = utils.get_cfn_stack_outputs(STACK_NAME, 'MLModelBucketName')
athena_query_results_bucket_name = utils.get_cfn_stack_outputs(STACK_NAME, 'AthenaQueryResultsBucketName')
feature_store_bucket_name = utils.get_cfn_stack_outputs(STACK_NAME, 'FeatureStoreBucketName')
hotel_cluster_prediction_fn_arn = utils.get_cfn_stack_outputs(STACK_NAME, 'HotelClusterPredictionFunction')
hotel_cluster_prediction_ddb_table_name = utils.get_cfn_stack_outputs(STACK_NAME, 'HotelClusterPredictionsTableName')

logger.info(f"data_bucket_name={data_bucket_name},\nathena_query_results_bucket_name={athena_query_results_bucket_name},\n"
            f"model_bucket_name={model_bucket_name}\nfeature_store_bucket_name={feature_store_bucket_name},\n"
            f"hotel_cluster_prediction_fn_arn={hotel_cluster_prediction_fn_arn}\nhotel_cluster_prediction_ddb_table_name={hotel_cluster_prediction_ddb_table_name}")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
2022-06-10 02:32:24,829,<ipython-input-178-4992a499e393>,<module>,10,INFO,p7027,data_bucket_name=expedia-customer-behavior-data-195cbf60,
athena_query_results_bucket_name=athena-query-results-195cbf60,
model_bucket_name=expedia-ml-models-195cbf60
feature_store_bucket_name=expedia-feature-store-offline-195cbf60,
hotel_cluster_prediction_fn_arn=arn:aws:lambda:us-east-1:015469603702:function:PredictHotelCluster
hotel_cluster_prediction_ddb_table_name=ExpediaPerCustomerHotelClusterPredictions


In [179]:
# read outputs from previous notebooks that are needed by this notebook.
# these are available as local files.
customer_inputs_fg_name = utils.read_param("customer_inputs_fg_name")
destinations_fg_name = utils.read_param("destinations_fg_name")
customer_inputs_fg_table = utils.read_param("customer_inputs_fg_table")
destinations_fg_table = utils.read_param("destinations_fg_table")
customer_inputs_fg_name = utils.read_param("customer_inputs_fg_name")
ml_model_endpoint_name = utils.read_param("endpoint_name")

# read params from the cloud formation stack
raw_data_dir = utils.get_cfn_stack_parameters(STACK_NAME, 'RawDataDir')
app_name = utils.get_cfn_stack_parameters(STACK_NAME, 'AppName')

training_dataset_fname = utils.get_cfn_stack_parameters(STACK_NAME, 'TrainingDatasetFileName')
test_dataset_fname = utils.get_cfn_stack_parameters(STACK_NAME, 'TestDatasetFileName')
validation_dataset_fname = utils.get_cfn_stack_parameters(STACK_NAME, 'ValidationDatasetFileName')

training_job_instance_type = utils.get_cfn_stack_parameters(STACK_NAME, 'TrainingJobInstanceType')
if training_job_instance_type is None:
    training_job_instance_type = "ml.m5.xlarge"
training_job_instance_count = int(utils.get_cfn_stack_parameters(STACK_NAME, 'TrainingJobNodeInstanceCount'))

model_ep_instance_type = utils.get_cfn_stack_parameters(STACK_NAME, 'ModelEndpointInstanceType')
model_ep_instance_count = int(utils.get_cfn_stack_parameters(STACK_NAME, 'ModelEndpointInstanceCount'))

customer_input_stream_name = utils.get_cfn_stack_parameters(STACK_NAME, 'CustomerInputStreamName')
            
logger.info(f"customer_inputs_fg_table={customer_inputs_fg_table},\ndestinations_fg_table={destinations_fg_table},\n"
            f"customer_inputs_fg_name={customer_inputs_fg_name},\ndestinations_fg_name={destinations_fg_name}\n"
            f"raw_data_dir={raw_data_dir},\ntraining_dataset_fname={training_dataset_fname},\n"
            f"test_dataset_fname={test_dataset_fname},\nvalidation_dataset_fname=-{validation_dataset_fname}\n"
            f"training_job_instance_type={training_job_instance_type},\ntraining_job_instance_count={training_job_instance_count},\n"
            f"model_ep_instance_type={model_ep_instance_type},\nmodel_ep_instance_count={model_ep_instance_count},\ncustomer_input_stream_name={customer_input_stream_name}")

2022-06-10 02:32:24,847,utils.py,read_param,130,INFO,p7027,read_param, fpath=../config/customer_inputs_fg_name, read customer_inputs_fg_name=expedia-customer-inputs-2022-6-8-15-0
2022-06-10 02:32:24,848,utils.py,read_param,130,INFO,p7027,read_param, fpath=../config/destinations_fg_name, read destinations_fg_name=expedia-destinations-2022-6-8-15-0
2022-06-10 02:32:24,849,utils.py,read_param,130,INFO,p7027,read_param, fpath=../config/customer_inputs_fg_table, read customer_inputs_fg_table=expedia-customer-inputs-2022-6-8-15-0-1654700956
2022-06-10 02:32:24,851,utils.py,read_param,130,INFO,p7027,read_param, fpath=../config/destinations_fg_table, read destinations_fg_table=expedia-destinations-2022-6-8-15-0-1654701052
2022-06-10 02:32:24,851,utils.py,read_param,130,INFO,p7027,read_param, fpath=../config/customer_inputs_fg_name, read customer_inputs_fg_name=expedia-customer-inputs-2022-6-8-15-0
2022-06-10 02:32:24,853,utils.py,read_param,130,INFO,p7027,read_param, fpath=../config/endpoint_n

## Update the lambda function

The lambda function handler for the Kinesis stream needs to be updated with the SageMaker endpoint name (this name was not available at the time of deploying the Lambda via the cloud formation template).

In [180]:
# clients for the services we are going to use
lambda_client = boto3.client('lambda')

In [181]:
# we use the environment variables for the lambda as a mechanism for passing config values. 
# Sagemaker endpoint name, the destinations feature group name etc are all available 
# as variables in this notebook (read already from local files). In a production environment
# these would be read from Parameter Store.

logger.info(f'updating Lambda function with ARN={hotel_cluster_prediction_fn_arn} to use ML model endpoint: {ml_model_endpoint_name}')
variables = lambda_client.get_function_configuration(FunctionName=hotel_cluster_prediction_fn_arn)['Environment']['Variables']
variables['ENDPOINT_NAME'] = ml_model_endpoint_name
variables['FG_NAME'] = destinations_fg_name
variables['DDB_TABLE_NAME'] = hotel_cluster_prediction_ddb_table_name
variables['ONLINE_FEATURE_GROUP_KEY'] = 'srch_destination_id'
variables['ONLINE_FEATURE_GROUP_FEATURES_OF_INTEREST'] = DESTINATION_FEATURES
variables['PREDICTED_VARIABLE'] = PREDICTED_VARIABLE

resp = lambda_client.update_function_configuration(
    FunctionName=hotel_cluster_prediction_fn_arn,
      Environment={
        'Variables': variables
    }
)

2022-06-10 02:32:26,025,<ipython-input-181-073dbc3e9a62>,<module>,1,INFO,p7027,updating Lambda function with ARN=arn:aws:lambda:us-east-1:015469603702:function:PredictHotelCluster to use ML model endpoint: hotel-cluster-prediction-ml-model-2022-06-08-19-12-46-266


## Stream test data
At this point we are all set to stream test data on the Kinesis data stream. This test data is available in a local file.

In [182]:
# read test data from local file
fpath = os.path.join(LOCAL_DATA_DIR, test_dataset_fname)
df = pd.read_csv(fpath)
logger.info(f"read test data from {fpath}, dataframe shape is {df.shape}")
df.head()

2022-06-10 02:32:26,259,<ipython-input-182-0cf1294fdcac>,<module>,4,INFO,p7027,read test data from ../data/test.csv, dataframe shape is (2009, 22)


,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,duration,days_to_trip,start_of_trip_weekend,end_of_trip_weekend
0,2,3,66,318,8031,1850.5388,838999,0,0,0,...,1,8278,1,2,50,368,2.0,225.0,0,1
1,2,3,66,351,8978,122.1979,480943,0,0,9,...,1,35125,4,2,50,1056,1.0,8.0,0,1
2,2,3,66,331,36172,69.9082,96493,0,0,1,...,2,11319,1,2,50,681,2.0,8.0,0,1
3,2,3,66,226,51467,136.4249,958118,0,0,9,...,1,30872,4,2,50,350,1.0,4.0,0,1
4,2,3,66,254,23820,1459.1560,318859,0,0,0,...,1,60915,4,4,8,110,18.0,141.0,0,1


In [183]:
# print out the first record just for debug purposes
record = json.loads(df.to_json(orient='records'))[0]
record

{'site_name': 2,
 'posa_continent': 3,
 'user_location_country': 66,
 'user_location_region': 318,
 'user_location_city': 8031,
 'orig_destination_distance': 1850.5388,
 'user_id': 838999,
 'is_mobile': 0,
 'is_package': 0,
 'channel': 0,
 'srch_adults_cnt': 2,
 'srch_children_cnt': 4,
 'srch_rm_cnt': 1,
 'srch_destination_id': 8278,
 'srch_destination_type_id': 1,
 'hotel_continent': 2,
 'hotel_country': 50,
 'hotel_market': 368,
 'duration': 2.0,
 'days_to_trip': 225.0,
 'start_of_trip_weekend': 0,
 'end_of_trip_weekend': 1}

In [184]:
# stream each row of the dataframe as a json to the Kinesis data stream
kinesis_client = boto3.client('kinesis')

for record in json.loads(df.to_json(orient='records'))[:RECORDS_TO_STREAM]:
    data = json.dumps(record)
    logger.info(f"Sending data, record.{PK}={record[PK]}...")
    response = kinesis_client.put_record(StreamName = customer_input_stream_name,
                                         Data = data,
                                         PartitionKey = PK)

    if (response['ResponseMetadata']['HTTPStatusCode'] != 200):
        logger.error("ERROR: Kinesis put_record failed: \n{}".format(json.dumps(response)))
    else:
        logger.info("data sent successfully...")

Sending data, record.user_id=838999...
data sent successfully...
Sending data, record.user_id=480943...
data sent successfully...
Sending data, record.user_id=96493...
data sent successfully...
Sending data, record.user_id=958118...
data sent successfully...
Sending data, record.user_id=318859...
data sent successfully...


## Check the DynamoDB table

Now check the DynamoDB table (ExpediaPerCustomerHotelClusterPredictionsunless changed when deploying the Cloud Formation template) for new data inserted corresponding to the records streamed in the previous step. Look for the hotel_cluster_predicted field, this field contains the prediction from the SageMaker model endpoint.

Also check the logs of the Lambda function to see that it got invokes for every record put on the Kinesis stream.

<img src="../images/ddb_table.png">Data Profile</img>